In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_pickle('shared/Project-3_NYC_311_Calls.pkl')

In [3]:
df = df.set_index(pd.DatetimeIndex(df['Created Date']))

del df['Created Date']

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 33780977 entries, 2011-04-06 00:00:00 to 2011-04-06 00:00:00
Data columns (total 11 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   Unique Key              int64 
 1   Agency                  object
 2   Agency Name             object
 3   Complaint Type          object
 4   Descriptor              object
 5   Location Type           object
 6   Incident Zip            object
 7   City                    object
 8   Resolution Description  object
 9   Borough                 object
 10  Open Data Channel Type  object
dtypes: int64(1), object(10)
memory usage: 3.0+ GB


In [5]:
df.head()

,Unique Key,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,City,Resolution Description,Borough,Open Data Channel Type
Created Date,,,,,,,,,,,
2011-04-06 00:00:00,20184537,HPD,Department of Housing Preservation and Develop...,HEATING,HEAT,RESIDENTIAL BUILDING,10002.0,NEW YORK,More than one complaint was received for this ...,MANHATTAN,UNKNOWN
2011-04-06 00:00:00,20184538,HPD,Department of Housing Preservation and Develop...,GENERAL CONSTRUCTION,WINDOWS,RESIDENTIAL BUILDING,11236.0,BROOKLYN,The Department of Housing Preservation and Dev...,BROOKLYN,UNKNOWN
2011-04-06 00:00:00,20184539,HPD,Department of Housing Preservation and Develop...,PAINT - PLASTER,WALLS,RESIDENTIAL BUILDING,10460.0,BRONX,The Department of Housing Preservation and Dev...,BRONX,UNKNOWN
2022-07-08 11:14:43,54732265,DSNY,Department of Sanitation,Dirty Condition,Trash,Sidewalk,10467.0,BRONX,The Department of Sanitation investigated this...,BRONX,PHONE
2011-04-06 00:00:00,20184540,HPD,Department of Housing Preservation and Develop...,NONCONST,VERMIN,RESIDENTIAL BUILDING,10460.0,BRONX,The Department of Housing Preservation and Dev...,BRONX,UNKNOWN


In [6]:
df.describe()

,Unique Key
count,3.378098e+07
mean,3.802665e+07
std,1.209574e+07
min,1.056422e+07
25%,2.793121e+07
50%,3.798303e+07
75%,4.830951e+07
max,5.840682e+07


### Q1

In [7]:
# Filter data for the year 2022
df_2022 = df[df.index.year == 2022]

# Resample the data to get daily counts of unique complaints
daily_complaints_2022 = df_2022['Unique Key'].resample('D').count()

# Calculate the average number of daily complaints
avg_daily_complaints_2022 = daily_complaints_2022.mean()

# Print the result
avg_daily_complaints_2022

8684.320547945206

### Q2

In [11]:
# Resample the data to get daily counts of unique complaints
daily_complaints = df['Unique Key'].resample('D').count()

# Find the date with the maximum number of complaints
date_of_max_complaints = daily_complaints.idxmax()

# Print the result
print(date_of_max_complaints.strftime('%Y-%m-%d'))


2020-08-04


### Q3

In [13]:
# On the date the maximum number of calls were received, what was the most important complaint type?
# Find the date with the maximum number of calls
max_calls_date = df['Unique Key'].resample('D').count().idxmax()

# Filter the dataset for that specific date and find the most common complaint type
most_common_complaint = df.loc[df.index.date == max_calls_date.date()]['Complaint Type'].value_counts().idxmax()
print(most_common_complaint)

Damaged Tree


### Q4

In [15]:
# Quietest Month historically
quietest_month = df['Unique Key'].resample('M').count().idxmin().strftime("%b")
print(quietest_month)

/tmp/ipykernel_4756/3140665661.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  quietest_month = df['Unique Key'].resample('M').count().idxmin().strftime("%b")


Aug


### Q5

In [17]:
# Resample your time series to a daily frequency.  Perform ETS decomposition based on an additive model.  What is the value of the seasonal component on 2020-12-25
from statsmodels.tsa.seasonal import seasonal_decompose

# Resample the series to a daily frequency
daily_complaints = df['Unique Key'].resample('D').count()

# Perform ETS decomposition based on an additive model
result = seasonal_decompose(daily_complaints, model='additive')

# Get the seasonal component value on 2020-12-25, rounded to the nearest integer
seasonal_component = result.seasonal['2020-12-25'].round()
print(seasonal_component)

183.0


### Q6

In [19]:
# Resample the data to daily frequency, summing the number of complaints each day
daily_complaints = df['Unique Key'].resample('D').count()

# Calculate autocorrelation for lag of 1
autocorrelation_lag_1 = daily_complaints.autocorr(lag=1)

# Print the autocorrelation
print(autocorrelation_lag_1)

0.7517059728398577


### Q7

In [20]:
from prophet import Prophet
from sklearn.metrics import mean_squared_error
from math import sqrt

# Prepare the data in the format required by Prophet
df_prophet = daily_complaints.reset_index()
df_prophet.columns = ['ds', 'y']

# Split the data into training and test sets
train = df_prophet.iloc[:-90]
test = df_prophet.iloc[-90:]

# Initialize and fit the Prophet model
model = Prophet()
model.fit(train)

# Create a dataframe for future predictions
future = model.make_future_dataframe(periods=90)

# Make predictions
forecast = model.predict(future)

# Extract predictions corresponding to the test set
predictions = forecast.iloc[-90:]['yhat']

# Calculate RMSE
rmse = sqrt(mean_squared_error(test['y'], predictions))
print(rmse)

/opt/conda/envs/mggy8413/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
14:57:23 - cmdstanpy - INFO - Chain [1] start processing
14:57:26 - cmdstanpy - INFO - Chain [1] done processing


1233.7823321393885
